<a href="https://colab.research.google.com/github/wilmi94/MasterThesis-AE/blob/main/notebooks/sdo_ConvLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SDO/AIA-171A End-to-End ConvLSTM Model

> This notebook is part of the Master Thesis *Predicting Coronal Mass Ejections using Machine Learning methods* by Wilmar Ender, FH Wiener Neustadt, 2023.

This notebook is considered the final model of the thesis and therefore includes the parts Data Preparation (section 4.4 in thesis), Model (section 4.5 in thesis) and Evaluation (section 4.6 in thesis).

**Objective:** \\
This notebook aims to perform the prediction of CMEs based on a given sequence of full-disk images from the SDO/AIA.

**Solar event list:** \\
*Liu et al. 2020, Predicting Coronal Mass Ejections Using SDO/HMI Vector Magnetic Data Products and Recurrent Neural Networks*

**Dataset:** \\
*Ahmadzadeh et al. 2019, A Curated Image Parameter Data Set from the Solar Dynamics Observatory Mission*. \\
Accessed via *sdo-cli* (https://github.com/i4Ds/sdo-cli)

*Predicting Solar Flares Using a Long Short-term Memory Network. Liu, H., Liu, C., Wang, J. T. L., Wang, H., ApJ., 877:121, 2019.*


...

## Setting up the Notebook

In [43]:
%%capture
pip install -U sdo-cli

In [44]:
import pandas as pd
import csv
import sys
import numpy as np
import cv2
import os
import shutil
from datetime import datetime
import subprocess
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive

#from keras.utils import np_utils
from keras.models import *
from keras.layers import *

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

try :
    import tensorflow as tf
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
except Exception as e:
    print('turn off loggins is not supported')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
from keras.models import *
from keras.layers import *
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [45]:
# mount Google drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Change present working directory
%cd /content/drive/MyDrive/Academia/MSc. Aerospace Engineering - FH Wiener Neustadt/4. Master Thesis/03-Work/

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-46-ceace3ed6187>", line 2, in <cell line: 2>
    get_ipython().run_line_magic('cd', '/content/drive/MyDrive/Academia/MSc. Aerospace Engineering - FH Wiener Neustadt/4. Master Thesis/03-Work/')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recen

In [46]:
# check content
!ls -a

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ls: cannot open directory '.': Transport endpoint is not connected


## Helper Functions
Define helper-functions to either speed up the calculation process of for better automation.

In [47]:
def rename_files_in_folder(folder_path):
  """
    Rename files in the specified directory by removing the "__171" substring from their names.

    Parameters:
    - folder_path (str): The path to the directory containing the files to be renamed.

    If the provided path is not a directory, a message will be printed, and no renaming will occur.

    Example:
    >>> rename_files_in_folder("/path/to/directory")
    Renamed: example__171.txt to example.txt
    Renamed: image__171.png to image.png
    ...
    """

    # Check if the provided path is a directory
  if not os.path.isdir(folder_path):
      print(f"{folder_path} is not a directory.")
      return

  # Iterate through files in the directory
  for filename in os.listdir(folder_path):
      old_file_path = os.path.join(folder_path, filename)

      # Check if it's a file (not a directory)
      if os.path.isfile(old_file_path):
          # Split the filename and extension
          base_name, file_extension = os.path.splitext(filename)

          # Check if the filename contains "__171"
          if "__171" in base_name:
              # Remove the "__171" part
              new_base_name = base_name.replace("__171", "")
              new_filename = f"{new_base_name}{file_extension}"
              new_file_path = os.path.join(folder_path, new_filename)

              # Rename the file
              os.rename(old_file_path, new_file_path)
              print(f"Renamed: {filename} to {new_filename}")


In [48]:
def compare_filenames_with_dataframe(directory, dataframe):
  """
    Compare filenames in a directory with timestamps in a DataFrame.

    Parameters:
    - directory (str): The path to the directory containing files to be compared.
    - dataframe (pd.DataFrame): A DataFrame containing timestamps to be compared with filenames.

    Returns:
    - statistics (dict): A dictionary with the following statistics:
        - 'Total Directory Files': Total files in the directory.
        - 'Total DataFrame Names': Total timestamps in the DataFrame.
        - 'Common Files': Number of files with corresponding timestamps.
        - 'Missing Files': Number of timestamps without corresponding files.
        - 'Extra Files': Number of files without corresponding timestamps.
    - df_missing (pd.DataFrame): A DataFrame containing missing timestamps in a more readable format.

    This function compares filenames in the specified directory with timestamps in the given DataFrame.
    It calculates and returns statistics on common, missing, and extra files, along with a DataFrame of missing timestamps.

    Example:
    >>> statistics, df_missing = compare_filenames_with_dataframe("/path/to/directory", df)
    Total Directory Files: 50
    Total DataFrame Names: 60
    Common Files: 40
    Missing Files: 20
    Extra Files: 10
    # 'statistics' dictionary and 'df_missing' DataFrame are returned
    """

  file_end = '.jpeg'
  # Get list of filenames from the directory
  directory_filenames = [filename for filename in os.listdir(directory) if filename.endswith(file_end)]

  # Get list of names from the DataFrame
  df_check = dataframe.copy()
  df_check['Timestamp'] = pd.to_datetime(df_check['Timestamp']).dt.strftime('%Y-%m-%dT%H%M%S.jpeg') # Warning! wavelength is now hard coded

  dataframe_names = df_check['Timestamp'].tolist()  # the name of the image should correspond to the timestamp

  # Compare filenames
  common_filenames = set(directory_filenames) & set(dataframe_names)
  missing_filenames = set(dataframe_names) - set(directory_filenames)
  extra_filenames = set(directory_filenames) - set(dataframe_names)

  df_missing = pd.DataFrame (data = missing_filenames,  columns=['Timestamp'])
  df_missing['Timestamp'] = df_missing['Timestamp'].str.replace(r'.jpeg', '', regex=True)
  df_missing['Timestamp'] = pd.to_datetime(df_missing['Timestamp']).dt.strftime('%Y-%m-%dT%H:%M:%S')

  # Calculate statistics
  total_directory_files = len(directory_filenames)
  total_dataframe_names = len(dataframe_names)
  total_common_files = len(common_filenames)
  total_missing_files = len(missing_filenames)
  total_extra_files = len(extra_filenames)
  print('Total Directory Files: ', total_directory_files)
  print('Total DataFrame Names: ', total_dataframe_names)
  print('Common Files: ', total_common_files)
  print('Missing Files: ', total_missing_files)
  print('Extra Files: ', total_extra_files)

  statistics = {
      'Total Directory Files': total_directory_files,
      'Total DataFrame Names': total_dataframe_names,
      'Common Files': total_common_files,
      'Missing Files': total_missing_files,
      'Extra Files': total_extra_files,
      #'Common File Names': common_filenames,
      'Missing File Names': missing_filenames,
      'Extra File Names': extra_filenames
  }

  return statistics, df_missing

In [49]:
# import pandas as pd
# import os
# from datetime import datetime

def create_csv_from_directory(dataframe, directory_path, output_csv_path):
    # Initialize an empty list to store timestamp-label pairs
    timestamp_label_pairs = []

    # Use os.walk to recursively traverse the directory and its subfolders
    for root, _, files in os.walk(directory_path):
        for file_name in files:
            # Extract the timestamp from the file name
            parts = file_name.split('T')
            if len(parts) == 2:
                file_timestamp_str = parts[0] + 'T' + parts[1].split('.')[0]
            else:
                # Handle invalid file name format
                continue

            try:
                # Convert the file timestamp string to a datetime object
                file_timestamp = datetime.strptime(file_timestamp_str, '%Y-%m-%dT%H%M%S')
            except ValueError:
                # Handle invalid timestamp format
                continue

            # Find the corresponding row in the DataFrame based on the timestamp
            matching_row = dataframe[dataframe['Timestamp'] == file_timestamp.strftime('%Y-%m-%dT%H:%M:%S')]

            if not matching_row.empty:
                # Get the label from the matching row
                label = matching_row.iloc[0]['Label']

                # Append the timestamp-label pair to the list
                timestamp_label_pairs.append((file_timestamp_str, label))

    # Create a new DataFrame from the list of timestamp-label pairs
    result_df = pd.DataFrame(timestamp_label_pairs, columns=['Timestamp', 'Label'])

    # Save the DataFrame to a CSV file
    result_df.to_csv(output_csv_path, index=False)



In [50]:
def copy_last_n_images(source_directory, destination_directory, n=None):
    # Get a list of files in the source directory
    files = sorted(os.listdir(source_directory))

    # Ensure the destination directory exists
    os.makedirs(destination_directory, exist_ok=True)

    # Copy the last n files, or all files if n is None
    files_to_copy = files[-n:] if n is not None else files

    for file_name in files_to_copy:
        source_path = os.path.join(source_directory, file_name)
        destination_path = os.path.join(destination_directory, file_name)
        shutil.copy2(source_path, destination_path)

In [51]:

def resize_images(source_dir, target_dir):
  # Function to resize images
    with tqdm(total=len(os.listdir(source_dir)), desc=f"Resizing {source_dir} images") as pbar:
        for filename in os.listdir(source_dir):
            if filename.endswith('.jpeg'):
                source_path = os.path.join(source_dir, filename)
                target_path = os.path.join(target_dir, filename)
                # Open the image and resize it
                img = Image.open(source_path)
                img = img.resize((new_width, new_height), Image.ANTIALIAS)
                img.save(target_path)
                pbar.update(1)

In [52]:
# Function to normalize images from 0 to 255 with a progress bar (no print messages)
def normalize_images(source_dir, target_dir, dataset_type):
    images = os.listdir(source_dir)
    with tqdm(total=len(images), desc=f"Normalizing {dataset_type} images", unit="image", dynamic_ncols=True) as pbar:
        for filename in images:
            if filename.endswith('.jpeg'):
                source_path = os.path.join(source_dir, filename)
                target_path = os.path.join(target_dir, filename)

                # Load the image
                img = cv2.imread(source_path)

                # Normalize the image from 0 to 255
                normalized_img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

                # Save the normalized image
                cv2.imwrite(target_path, normalized_img)

                # Update progress bar
                pbar.update(1)

In [53]:

def classify_and_move_images(source_dir, positive_dir, negative_dir):
  # Function to classify and move images based on timestamps within event ranges training
    for filename in os.listdir(source_dir):
        if filename.endswith('.jpeg'):
            timestamp = filename[:-5]  # Extract timestamp from the filename (remove '.jpeg' extension)
            # Check if the timestamp is within any event range in the 'cme_events' DataFrame
            for _, event in cme_events.iterrows():
                if(event['CME']!="no"):
                  start_time = event['Start Time']  # Replace 'start_time' with the actual column name
                  end_time = event['End Time']  # Replace 'end_time' with the actual column name
                  if start_time <= timestamp <= end_time:
                      # Move to the positive folder
                      shutil.move(os.path.join(source_dir, filename), os.path.join(positive_dir, filename))
                      break  # Move to the next image
            else:
                # If the timestamp does not fall within any event range, move to the negative folder
                shutil.move(os.path.join(source_dir, filename), os.path.join(negative_dir, filename))

In [54]:
def delete_files_with_pattern(directory, pattern):
    # Recursively traverse the directory tree
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(pattern):
                file_path = os.path.join(root, filename)
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Failed to delete {file_path}: {str(e)}")

In [55]:
def load_and_preprocess_images_with_time(directory, image_size=(64,64)):
    images = []
    labels = []
    timestamps = []  # To store timestamps extracted from filenames

    for dirname in os.listdir(directory):
        for filename in os.listdir(os.path.join(directory, dirname)):
            if filename.endswith('.jpeg'):
                image_path = os.path.join(directory, dirname, filename)
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)
                image = image.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1]

                images.append(image)

                # Extract timestamp from the filename
                timestamp_str = filename[:-5]  # Remove '.jpeg' extension
                timestamp = datetime.strptime(timestamp_str, "%Y-%m-%dT%H%M%S")
                timestamps.append(timestamp)

                # Assuming that positive examples have filenames containing "pos" and negative examples "neg"
                if "pos" in dirname:
                    labels.append(1)  # Positive example
                elif "neg" in dirname:
                    labels.append(0)  # Negative example

    return np.array(images), np.array(labels), np.array(timestamps)

In [56]:
# # ALTERNATIVE FUNCTION GPT

# def load_and_preprocess_images_with_time(directory, image_size=(new_width, new_height)):
#     images = []
#     labels = []
#     timestamps = []

#     image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
#         rescale=1./255,
#         samplewise_center=False,
#         samplewise_std_normalization=False,
#         validation_split=0.2
#     )

#     image_dataset = image_data_generator.flow_from_directory(
#         directory,
#         target_size=image_size,
#         batch_size=32,
#         class_mode='binary',  # Change this based on your problem (binary or categorical)
#         subset='training'
#     )

#     for image_batch, labels_batch in image_dataset:
#         images.extend(image_batch)
#         labels.extend(labels_batch)

#         # Extract timestamp from the filenames in the image_dataset
#         timestamps.extend([
#             datetime.strptime(os.path.splitext(os.path.basename(fname))[0], "%Y-%m-%dT%H%M%S")
#             for fname in image_dataset.filepaths
#         ])

#     return np.array(images), np.array(labels), np.array(timestamps)


## Load Event-Lists
All lists are taken from *Liu et al. 2020*.

These are:
* a base/initail event list, which holds all the events and non-events
* a sampled training list
* a sampled testing list




### Base Event List
The follwoing initial list holds all events and non-events from the study.

In [ ]:
# load list with both (neg, pos) labels
df_cme_list = pd.read_csv(r'00_Dataset/event_lists/all_cme_events.csv', delimiter =';')

# convert time stamp such that sdo-cli can read them
df_cme_list['Start Time'] = pd.to_datetime(df_cme_list['Start Time']).dt.strftime('%Y-%m-%dT%H:%M:%S')
df_cme_list['Peak Time'] = pd.to_datetime(df_cme_list['Peak Time']).dt.strftime('%Y-%m-%dT%H:%M:%S')
df_cme_list['End Time'] = pd.to_datetime(df_cme_list['End Time']).dt.strftime('%Y-%m-%dT%H:%M:%S')
df_cme_list.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-57-379aa9343459>", line 2, in <cell line: 2>
    df_cme_list = pd.read_csv(r'00_Dataset/event_lists/all_cme_events.csv', delimiter =';')
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 950, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 605, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 1

The next lists are divided into a testing and training list. These list contain the above lists. In particular, the training list contain the samples from the years 2010-2014, while the testing list holds the samples from the yeas 2015-2019. Hence the lists and following datsets are disjoint. Furtermore, the events were sampled for predicting CMEs within the next x hours (x = 12, 24, 36, 48 or 60). In the present case x = 12 hours.

These lists hold the following infromation:

> N means there is a >=M class flare within the next x hours but this flare is not associated with a CME. \\
> P means there is a >=M class flare within the next x hours and this flare is associated with a CME. \\
> The second column holds the Timestamp of the sample. \\
> The third column and fourth column are titled NOAA active region number and HARP number, respectively. \\
> Starting from the fifth column, you can see physical parameters of data samples, which include 18 SHARP parameters:
TOTUSJH, TOTPOT, TOTUSJZ, ABSNJZH, SAVNCPP, USFLUX, AREA_ACR, MEANPOT, R_VALUE, SHRGT45, MEANGAM, MEANJZH, MEANGBT, MEANGBZ, MEANJZD, MEANGBH, MEANSHR, MEANALP.

#### Training List


In [ ]:
# load list with both (neg, pos) labels of training list for t= 12 h prediction window
df_cme_training_12 = pd.read_csv(r'00_Dataset/event_lists/Liu2020_CME_data_samples/normalized_training_12.csv', delimiter =',')
print('The training list contains ', df_cme_training_12.shape[0], 'samples')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-57-1c42ae118620>", line 2, in <cell line: 2>
    df_cme_training_12 = pd.read_csv(r'00_Dataset/event_lists/Liu2020_CME_data_samples/normalized_training_12.csv', delimiter =',')
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 950, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 605, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.10/dist-packag

In [57]:
df_cme_training_12.head()

,Label,Timestamp,NOAA_AR_NUM,HARP_NUM,TOTUSJH,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,...,R_VALUE,SHRGT45,MEANGAM,MEANJZH,MEANGBT,MEANGBZ,MEANJZD,MEANGBH,MEANSHR,MEANALP
0,N,2010-05-05T05:22:22.90Z,11069,8,-0.5461,-0.5672,-0.6696,0.3159,0.2083,-0.7610,...,0.3751,-0.4109,-0.1049,-3.3554,0.5327,0.6921,-0.1673,0.5768,-0.1024,-1.1464
1,N,2010-05-05T05:34:22.90Z,11069,8,-0.5438,-0.5635,-0.6607,0.3360,0.2741,-0.7552,...,0.3969,-0.3188,-0.0283,-3.2984,0.4808,0.6302,-0.0497,0.5683,-0.0324,-1.1596
2,N,2010-05-05T05:46:22.90Z,11069,8,-0.5469,-0.5604,-0.6521,0.3315,0.3650,-0.7518,...,0.3961,-0.2149,0.0489,-3.1799,0.4268,0.5787,-0.1523,0.5588,0.0503,-1.1509
3,N,2010-05-05T06:22:22.90Z,11069,8,-0.5275,-0.5546,-0.6356,0.3042,0.2075,-0.7346,...,0.3842,-0.2724,0.0144,-2.9791,0.4130,0.5345,-0.0549,0.5549,0.0166,-1.0816
4,N,2010-05-05T06:34:22.90Z,11069,8,-0.5212,-0.5481,-0.6294,0.3653,0.2579,-0.7322,...,0.3554,-0.2530,0.0585,-3.1240,0.4596,0.6073,0.1217,0.5626,0.0660,-1.1460


In [58]:
# reduce dataframe to relevant columns
df_cme_training_12 = df_cme_training_12.drop(['TOTUSJH',	'TOTPOT',	'TOTUSJZ',	'ABSNJZH',	'SAVNCPP',
                     'USFLUX', 'AREA_ACR', 'MEANPOT', 'R_VALUE', 'SHRGT45',
                     'MEANGAM', 'MEANJZH', 'MEANGBT', 'MEANGBZ', 'MEANJZD',
                     'MEANGBH', 'MEANSHR', 'MEANALP'], axis=1)


In [59]:
df_cme_training_12['Timestamp'] = pd.to_datetime(df_cme_training_12['Timestamp']).dt.strftime('%Y-%m-%dT%H:%M:%S')
df_cme_training_12.head()

,Label,Timestamp,NOAA_AR_NUM,HARP_NUM
0,N,2010-05-05T05:22:22,11069,8
1,N,2010-05-05T05:34:22,11069,8
2,N,2010-05-05T05:46:22,11069,8
3,N,2010-05-05T06:22:22,11069,8
4,N,2010-05-05T06:34:22,11069,8


In [60]:
# df_cme_training_12 = df_cme_training_12.tail(df_cme_training_12.shape[0] -282)
# df_cme_training_12

In [61]:
# get negative samples
df_cme_training_12_neg = df_cme_training_12.loc[df_cme_training_12['Label'] == 'N']
df_cme_training_12_neg = df_cme_training_12_neg.reset_index(drop=True)
print('There are', df_cme_training_12_neg.shape[0], 'negative samples in the training set.\n')
df_cme_training_12_neg.tail()

There are 16960 negative samples in the training set.



,Label,Timestamp,NOAA_AR_NUM,HARP_NUM
16955,N,2016-01-01T21:22:08,12473,6206
16956,N,2016-01-01T21:34:08,12473,6206
16957,N,2016-01-01T21:46:08,12473,6206
16958,N,2016-01-01T21:58:08,12473,6206
16959,N,2016-01-01T22:22:08,12473,6206


In [62]:
# get positive samples
df_cme_training_12_pos = df_cme_training_12.loc[df_cme_training_12['Label'] == 'P']
df_cme_training_12_pos = df_cme_training_12_pos.reset_index(drop = True)
print('There are', df_cme_training_12_pos.shape[0], 'positive samples in the training set.\n')
df_cme_training_12_pos.tail()

There are 3387 positive samples in the training set.



,Label,Timestamp,NOAA_AR_NUM,HARP_NUM
3382,P,2015-12-28T11:46:08,12473,6206
3383,P,2015-12-28T11:58:08,12473,6206
3384,P,2015-12-28T12:10:08,12473,6206
3385,P,2015-12-28T12:22:08,12473,6206
3386,P,2015-12-28T12:34:08,12473,6206


#### Testing List

In [ ]:
# load list with both (neg, pos) labels of testing list for t= 12 h
df_cme_test_12 = pd.read_csv(r'00_Dataset/event_lists/Liu2020_CME_data_samples/normalized_testing_12.csv', delimiter =',')
# df_cme_test_12.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-63-faae02649428>", line 2, in <cell line: 2>
    df_cme_test_12 = pd.read_csv(r'00_Dataset/event_lists/Liu2020_CME_data_samples/normalized_testing_12.csv', delimiter =',')
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 950, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 605, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/pa

In [ ]:
# reduce dataframe to relevant columns
df_cme_test_12 = df_cme_test_12.drop(['TOTUSJH',	'TOTPOT',	'TOTUSJZ',	'ABSNJZH',	'SAVNCPP',
                     'USFLUX', 'AREA_ACR', 'MEANPOT', 'R_VALUE', 'SHRGT45',
                     'MEANGAM', 'MEANJZH', 'MEANGBT', 'MEANGBZ', 'MEANJZD',
                     'MEANGBH', 'MEANSHR', 'MEANALP'], axis=1)
df_cme_test_12.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-63-8d0eec8e6103>", line 2, in <cell line: 2>
    df_cme_test_12 = df_cme_test_12.drop(['TOTUSJH',	'TOTPOT',	'TOTUSJZ',	'ABSNJZH',	'SAVNCPP',
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py", line 5399, in drop
    return super().drop(
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py", line 4505, in drop
    obj = obj._drop_axis(labels, axis, level=level, errors=errors)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py", line 4546, in _drop_axis
    new

In [63]:
# convert time stamp
# convert time stamp such that sdo-cli can read them
df_cme_test_12['Timestamp'] = pd.to_datetime(df_cme_test_12['Timestamp']).dt.strftime('%Y-%m-%dT%H:%M:%S')
df_cme_test_12.tail()

,Label,Timestamp,NOAA_AR_NUM,HARP_NUM
1307,N,2017-09-08T21:46:41,12673,7115
1308,N,2017-09-08T21:58:41,12673,7115
1309,N,2017-09-08T22:10:41,12673,7115
1310,N,2017-09-08T22:22:41,12673,7115
1311,N,2017-09-08T22:34:41,12673,7115


In [64]:
df_cme_test_12_neg = df_cme_test_12.loc[df_cme_test_12['Label'] == 'N']
df_cme_test_12_neg = df_cme_test_12_neg.reset_index(drop=True)
print('There are', df_cme_test_12_neg.shape[0], 'negative samples in the test set.\n')
df_cme_test_12_neg.tail()

There are 762 negative samples in the test set.



,Label,Timestamp,NOAA_AR_NUM,HARP_NUM
757,N,2017-09-08T21:46:41,12673,7115
758,N,2017-09-08T21:58:41,12673,7115
759,N,2017-09-08T22:10:41,12673,7115
760,N,2017-09-08T22:22:41,12673,7115
761,N,2017-09-08T22:34:41,12673,7115


In [65]:
df_cme_test_12_pos = df_cme_test_12.loc[df_cme_test_12['Label'] == 'P']
df_cme_test_12_pos = df_cme_test_12_pos.reset_index(drop=True)
print('There are', df_cme_test_12_pos.shape[0], 'positive samples in the test set.\n')
df_cme_test_12_pos.tail()

There are 550 positive samples in the test set.



,Label,Timestamp,NOAA_AR_NUM,HARP_NUM
545,P,2017-09-08T05:10:41,12673,7115
546,P,2017-09-08T05:22:41,12673,7115
547,P,2017-09-08T05:34:41,12673,7115
548,P,2017-09-08T05:46:41,12673,7115
549,P,2017-09-08T06:10:41,12673,7115


In [66]:
# Usage
# folder_path = '00_Dataset/Liu2020_sampled/test_12/pos/'  # Replace with your Google Drive folder path
# rename_files_in_folder(folder_path)

By now we have four event lists from the sampled dataset:


*   df_cme_test_12_neg >> 762
*   df_cme_test_12_pos >> 550
*   df_cme_training_12_neg >> 16678
*   df_cme_training_12_pos >> 3387 samples


## Download and Explore the Image-Data
After importing and exploring the relevant lists, the images can be downloaded.
> **Note:** most of the below code lines are commented out. Otherweise, we would start downloading the images everytime we start the notebook.

### Training
Download the negative samples.

In [67]:
# create_sdo_aia_dataset(output_dir = './data/Liu2020_sampled/train/neg/', start_idx=0,  event_list = df_cme_training_12_neg, dt = '10min', wavelength = '171')

After downloading the samples, we check them in terms of completeness:

In [ ]:
directory_path_training_12_neg = '00_Dataset/Liu2020_sampled/training/neg/'
result_training_12_neg, missing_files_training_neg = compare_filenames_with_dataframe(directory_path_training_12_neg, df_cme_training_12_neg)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-68-090075077ce1>", line 2, in <cell line: 2>
    result_training_12_neg, missing_files_training_neg = compare_filenames_with_dataframe(directory_path_training_12_neg, df_cme_training_12_neg)
  File "<ipython-input-48-64fd24880c16>", line 33, in compare_filenames_with_dataframe
    directory_filenames = [filename for filename in os.listdir(directory) if filename.endswith(file_end)]
OSError: [Errno 107] Transport endpoint is not connected: '00_Dataset/Liu2020_sampled/training/neg/'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attr

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-68-090075077ce1>", line 2, in <cell line: 2>
    result_training_12_neg, missing_files_training_neg = compare_filenames_with_dataframe(directory_path_training_12_neg, df_cme_training_12_neg)
  File "<ipython-input-48-64fd24880c16>", line 33, in compare_filenames_with_dataframe
    directory_filenames = [filename for filename in os.listdir(directory) if filename.endswith(file_end)]
OSError: [Errno 107] Transport endpoint is not connected: '00_Dataset/Liu2020_sampled/training/neg/'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attr

In [68]:
result_training_12_neg['Common Files']

15361

As one can observe almost 500 images/samples couldn't be downloaded!

Next, we try to download the positive training samples

In [69]:
# create_sdo_aia_dataset(output_dir = './data/Liu2020_sampled/train/pos/', start_idx=0, event_list = df_cme_training_12_pos, dt = '10min', wavelength = '171')

Again, lets check the missing files:

In [ ]:
directory_path_training_12_pos = '00_Dataset/Liu2020_sampled/training/pos/'
result_training_12_pos, missing_files_training_pos = compare_filenames_with_dataframe(directory_path_training_12_pos, df_cme_training_12_pos)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-70-5cabffda6957>", line 2, in <cell line: 2>
    result_training_12_pos, missing_files_training_pos = compare_filenames_with_dataframe(directory_path_training_12_pos, df_cme_training_12_pos)
  File "<ipython-input-48-64fd24880c16>", line 33, in compare_filenames_with_dataframe
    directory_filenames = [filename for filename in os.listdir(directory) if filename.endswith(file_end)]
OSError: [Errno 107] Transport endpoint is not connected: '00_Dataset/Liu2020_sampled/training/pos/'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attr

In [70]:
total_training_files = result_training_12_pos['Common Files'] + result_training_12_neg['Common Files']
print('There are in total', total_training_files, 'training files.')

There are in total 18505 training files.


### Testing
Same for the negative samples for testing:

In [71]:
# create_sdo_aia_dataset(output_dir = './data/Liu2020_sampled/test_12/neg/', start_idx=0, event_list = df_cme_test_12_neg, dt = '10min', wavelength = '171')

In [ ]:
directory_path_test_12_neg = '00_Dataset/Liu2020_sampled/testing/neg/'
result_test_12_neg, missing_files_test_neg = compare_filenames_with_dataframe(directory_path_test_12_neg, df_cme_test_12_neg)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-72-e0b943bf63d1>", line 2, in <cell line: 2>
    result_test_12_neg, missing_files_test_neg = compare_filenames_with_dataframe(directory_path_test_12_neg, df_cme_test_12_neg)
  File "<ipython-input-48-64fd24880c16>", line 33, in compare_filenames_with_dataframe
    directory_filenames = [filename for filename in os.listdir(directory) if filename.endswith(file_end)]
OSError: [Errno 107] Transport endpoint is not connected: '00_Dataset/Liu2020_sampled/testing/neg/'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_tr

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-72-e0b943bf63d1>", line 2, in <cell line: 2>
    result_test_12_neg, missing_files_test_neg = compare_filenames_with_dataframe(directory_path_test_12_neg, df_cme_test_12_neg)
  File "<ipython-input-48-64fd24880c16>", line 33, in compare_filenames_with_dataframe
    directory_filenames = [filename for filename in os.listdir(directory) if filename.endswith(file_end)]
OSError: [Errno 107] Transport endpoint is not connected: '00_Dataset/Liu2020_sampled/testing/neg/'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_tr

...and for the positive samples:

In [72]:
# create_sdo_aia_dataset(output_dir = './data/Liu2020_sampled/test_12/pos/', start_idx= 0, event_list = df_cme_test_12_pos, dt = '10min', wavelength = '171')

In [ ]:
directory_path_test_12_pos = '00_Dataset/Liu2020_sampled/testing/pos/'
result_test_12_pos, missing_test_pos = compare_filenames_with_dataframe(directory_path_test_12_pos, df_cme_test_12_pos)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-73-6f750920da3f>", line 2, in <cell line: 2>
    result_test_12_pos, missing_test_pos = compare_filenames_with_dataframe(directory_path_test_12_pos, df_cme_test_12_pos)
  File "<ipython-input-48-64fd24880c16>", line 33, in compare_filenames_with_dataframe
    directory_filenames = [filename for filename in os.listdir(directory) if filename.endswith(file_end)]
OSError: [Errno 107] Transport endpoint is not connected: '00_Dataset/Liu2020_sampled/testing/pos/'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_tracebac

In [73]:
total_testing_files = result_test_12_neg['Common Files'] + result_test_12_pos['Common Files']
print('There are ', total_testing_files, ' total testing files' )

There are  1288  total testing files


### Update Event Lists

Since not all data samples could be downloaded, the event lists have to be adjusted acordingly.

In [74]:
train_dir_path = '00_Dataset/Liu2020_sampled/training'
output_train_csv_path = '00_Dataset/Liu2020_sampled/training/train_label.csv'

#create_csv_from_directory(df_cme_training_12, train_dir_path , output_train_csv_path)

In [ ]:
train_label_csv = pd.read_csv(r'00_Dataset/Liu2020_sampled/training/train_label.csv', delimiter =',')
train_label_csv

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-75-4aa304bfaba1>", line 1, in <cell line: 1>
    train_label_csv = pd.read_csv(r'00_Dataset/Liu2020_sampled/training/train_label.csv', delimiter =',')
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 950, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 605, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/reade

In [37]:
test_dir_path = '00_Dataset/Liu2020_sampled/testing'
output_test_csv_path = '00_Dataset/Liu2020_sampled/testing/test_label.csv'

#create_csv_from_directory(df_cme_test_12, test_dir_path, output_test_csv_path)
test_label_csv = pd.read_csv(r'00_Dataset/Liu2020_sampled/testing/test_label.csv', delimiter =',')
test_label_csv

,Timestamp,Label
0,2016-02-11T225809,N
1,2016-02-11T231009,N
2,2016-02-11T232209,N
3,2016-02-11T233410,N
4,2016-02-11T234610,N
...,...,...
1641,2016-02-14T173410,N
1642,2016-02-14T174610,N
1643,2016-02-14T182210,N
1644,2016-02-15T022210,N


**Summary:**
* Out of the possible 20347 training files, XXX could be downloaded.
* Out of the possible 20347 testing files, XXX could be downloaded.
* in detail, this results in the following split:
  * Training:
    * positive:
    * negative:
  * Testing:
    * positive:
    * negative:

## Make a reduced Dataset

Training:

In [39]:
working_dir = './03_sdo_ConvLSTM/'

In [40]:
# Example usage:
source_directory_training = '00_Dataset/Liu2020_sampled/training/neg'
destination_directory_training = '00_Dataset/Liu2020_sampled/training/all' #working_dir + 'data/testing'

n = None  # Replace with the desired number of last images to copy, or set to None to copy all files

copy_last_n_images(source_directory_training, destination_directory_training, n)

Testing:

In [41]:
# Example usage:
source_directory_testing = '00_Dataset/Liu2020_sampled/testing/pos'
destination_directory_testing =  '00_Dataset/Liu2020_sampled/testing/all' #working_dir + 'data/testing'

n = None  # Replace with the desired number of last images to copy, or set to None to copy all files

copy_last_n_images(source_directory_testing, destination_directory_testing, n)

## Display a Sample Image-Series

## Data Preparation


### Resize Image

In [ ]:
# Define the image size for resizing
new_width = 128
new_height = 128

In [ ]:
# Define the source and target directories
# source_directory_training = working_dir + 'data/training'
# source_directory_testing =  working_dir + 'data/testing'

source_directory_training_neg = './00_Dataset/Liu2020_sampled/training/neg'
source_directory_training_pos = './00_Dataset/Liu2020_sampled/training/pos'

source_directory_testing_neg  = './00_Dataset/Liu2020_sampled/testing/neg'
source_directory_testing_pos  = './00_Dataset/Liu2020_sampled/testing/pos'

target_directory_training =  working_dir + 'data/training_resized'
target_directory_testing =  working_dir + 'data/testing_resized'

# Create the target directories if they don't exist
os.makedirs(target_directory_training, exist_ok=True)
os.makedirs(target_directory_testing, exist_ok=True)

In [ ]:
# Resize testing images
print("Resizing testing")
resize_images(source_directory_testing_neg, target_directory_testing)
resize_images(source_directory_testing_pos, target_directory_testing)

In [ ]:
# Resize training images
print("Resizing training")
resize_images(source_directory_training_neg, target_directory_training)
resize_images(source_directory_training_pos, target_directory_training)

In [ ]:
# consider removing/deleting the previous files to free-up space
# !rm -r -f 03_sdo_ConvLSTM/PROTOTYPE/data/training
# !rm -r -f 03_sdo_ConvLSTM/PROTOTYPEdata/testing

### Normalize

In [ ]:
# Define the source and target directories
source_directory_training = target_directory_training # './03_sdo_ConvLSTM/PROTOTYPE/data/training_resized'
source_directory_testing =  target_directory_testing  #'./03_sdo_ConvLSTM/PROTOTYPE/data/testing_resized'

target_directory_training = working_dir + 'data/training_normalized'
target_directory_testing =  working_dir + 'data/testing_normalized'

# Create the target directories if they don't exist
os.makedirs(target_directory_training, exist_ok=True)
os.makedirs(target_directory_testing, exist_ok=True)

In [ ]:
# Normalize training images
normalize_images(source_directory_training, target_directory_training, "Training")

In [ ]:
# Normalize testing images
normalize_images(source_directory_testing, target_directory_testing, "Testing")

In [ ]:
print('There are', len(os.listdir(source_directory_training_neg)), 'files in the testing - negative class')
print('There are', len(os.listdir(source_directory_training_pos)), 'files in the testing - positive class')
print('In total:',  len(os.listdir(source_directory_training_pos )) + len(os.listdir(source_directory_training_neg)))

In [ ]:
# consider removing/deleting the previous files to free-up space
# !rm -r -f  03_sdo_ConvLSTM/PROTOTYPE/data/training_resized
# !rm -r -f  03_sdo_ConvLSTM/PROTOTYPE/data/testing_resized

In [ ]:
source_directory_testing_pos = './03_sdo_ConvLSTM/data/testing_resized/pos/'
target_directory_testing_pos = './03_sdo_ConvLSTM/data/testing_norm/pos/'

source_directory_testing_neg = './03_sdo_ConvLSTM/data/testing_resized/neg/'
target_directory_testing_neg = './03_sdo_ConvLSTM/data/testing_norm/neg/'

os.makedirs(target_directory_testing_pos, exist_ok=True)
os.makedirs(target_directory_testing_neg, exist_ok=True)

In [ ]:
print('There are', len(os.listdir(source_directory_testing_neg)), 'files in the testing - negative class')
print('There are', len(os.listdir(source_directory_testing_pos)), 'files in the testing - positive class')
print('In total:',  len(os.listdir(source_directory_testing_pos )) + len(os.listdir(source_directory_testing_neg)))

In [ ]:
# Normalize training images
normalize_images(source_directory_training_pos, target_directory_training_pos, "Training-pos")
normalize_images(source_directory_training_neg, target_directory_training_neg, "Training-neg")
# Normalize testing images
normalize_images(source_directory_testing_pos, target_directory_testing_pos, "Testing-pos")
normalize_images(source_directory_testing_neg, target_directory_testing_neg, "Testing-neg")
# !rm -r -f data/training_resized
# !rm -r -f data/testing_resized

### Classify training

In [ ]:
# consider removing/deleting the previous files to free-up space
# !rm -r -f 03_sdo_ConvLSTM/PROTOTYPE/processed_data/training
# !rm -r -f 03_sdo_ConvLSTM/PROTOTYPE/processed_data/testing

In [ ]:
# Define the paths to the training images and the target positive and negative folders
source_directory =     working_dir + 'data/training_normalized'
positive_destination = working_dir + 'processed_data/training/pos'
negative_destination = working_dir + 'processed_data/training/neg'

# Create the target directories if they don't exist
os.makedirs(positive_destination, exist_ok=True)
os.makedirs(negative_destination, exist_ok=True)

In [ ]:
# Get a list of filenames from the training directory
training_filenames = os.listdir(source_directory)

# Classify and move images
classify_and_move_images(source_directory, positive_destination, negative_destination)

print("Training classification completed.")


Testing

In [ ]:
# Define the paths to the testing images and the target positive and negative folders
source_directory =     working_dir + 'data/testing_normalized'
positive_destination = working_dir + 'processed_data/testing/pos'
negative_destination = working_dir + 'processed_data/testing/neg'

# Create the target directories if they don't exist
os.makedirs(positive_destination, exist_ok=True)
os.makedirs(negative_destination, exist_ok=True)

In [ ]:
# Get a list of filenames from the testing directory
testing_filenames = os.listdir(source_directory)

# Classify and move images
classify_and_move_images(source_directory, positive_destination, negative_destination)
print("Testing classification completed.")

In [ ]:
# consider removing/deleting the previous files to free-up space
# !rm -r -f 03_sdo_ConvLSTM/PROTOTYPE/data

In [ ]:
# training_directory = './03_sdo_ConvLSTM/data/training_resized/'
# file_pattern_to_delete = '__171 (1).jpeg'

# delete_files_with_pattern(training_directory, file_pattern_to_delete)

In [ ]:
# testing_directory = './03_sdo_ConvLSTM/data/testing_resized/'
# delete_files_with_pattern(testing_directory, file_pattern_to_delete)

### Create numpy data variable
Training

In [ ]:
training_directory = working_dir + 'processed_data/training'
#training_directory = working_dir + 'data/processed_data_256/training'

In [ ]:
# # ALTERNATIVE FUNCTION using keras....test it!

# Update your training and testing directories accordingly
training_directory = working_dir + 'processed_data/training'
testing_directory = working_dir +'processed_data/testing'

X_train, y_train, timestamps_train = load_and_preprocess_images_with_time(training_directory)
X_test, y_test, timestamps_test = load_and_preprocess_images_with_time(testing_directory)

In [ ]:
np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)

In [ ]:
#X_train, y_train, timestamps_train = load_and_preprocess_images_with_time(training_directory)

In [ ]:
X_train.size

In [ ]:
# Define the number of time steps (in this case, 10)
time_steps = 5

# Calculate the number of sequences
num_sequences = X_train.shape[0] - time_steps + 1

# Initialize an empty array to hold the sequences
X_train_sequences = np.empty((num_sequences, time_steps, new_width, new_height, 3))

# Create the sequences
for i in range(num_sequences):
    sequence = X_train[i:i +time_steps]
    X_train_sequences[i] = sequence

# Now, X_train_sequences has the shape (199, 10, 256, 256, 3)
X_train = X_train_sequences
y_train = y_train[:-time_steps+1]

In [ ]:
# recalc_train = False

# if recalc_test == True:
#   X_train, y_train, timestamps_train = load_and_preprocess_images_with_time(training_directory)

#   np.save('X_train_init.npy', X_train)
#   np.save('y_train_init.npy', y_train)
#   np.save('timestamps_train.npy', timestamps_train)

# else:
#   X_train = np.load('X_train_init.npy',  allow_pickle=True)
#   y_train = np.load('y_train_init.npy',  allow_pickle=True)
#   timestamps_test = np.load('timestamps_test_init.npy',  allow_pickle=True)

Testing

In [ ]:
#testing_directory  = working_dir + 'processed_data/testing'
testing_directory = working_dir + 'data/processed_data_256/testing'

In [ ]:
X_test, y_test, timestamps_test = load_and_preprocess_images_with_time(testing_directory)

In [ ]:
# Calculate the number of sequences
num_sequences = X_test.shape[0] - time_steps + 1

# Initialize an empty array to hold the sequences
X_test_sequences = np.empty((num_sequences, time_steps, new_width, new_height, 3))

# Create the sequences
for i in range(num_sequences):
    sequence = X_test[i:i + time_steps]
    X_test_sequences[i] = sequence

# Now, X_test_sequences has the shape (199, 10, 256, 256, 3)
X_test = X_test_sequences
y_test = y_test[:-time_steps+1]

In [ ]:
# file_pattern_to_delete = '__171 (1).jpeg'

# delete_files_with_pattern(training_directory, file_pattern_to_delete)

In [ ]:
# delete_files_with_pattern(testing_directory, file_pattern_to_delete)

In [ ]:
# recalc_test = False

# if recalc_test == True:
#   X_test, y_test, timestamps_test = load_and_preprocess_images_with_time(testing_directory)

#   np.save('X_test_init.npy', X_test)
#   np.save('y_test_init.npy', y_test)
#   np.save('timestamps_test_init.npy', timestamps_test)

# else:
#   X_test = np.load('X_test_init.npy', allow_pickle=True)
#   y_test = np.load('y_test_init.npy',  allow_pickle=True)
#   timestamps_test = np.load('timestamps_test_init.npy',  allow_pickle=True)

In [ ]:
# # Define the number of time steps (in this case, 10)
# time_steps = 10

# # Calculate the number of sequences
# num_sequences = X_train.shape[0] - time_steps + 1

# # Initialize a list to hold the sequences
# X_train_sequences = []

# # Create the sequences
# for i in range(num_sequences):
#     sequence = X_train[i:i + time_steps]
#     X_train_sequences.append(sequence)

# # Convert the list to a numpy array
# X_train = np.array(X_train_sequences)
# y_train = y_train[:-time_steps+1]


In [ ]:
# Define the number of time steps (in this case, 10)
# Calculate the number of sequences
num_sequences = X_test.shape[0] - time_steps + 1

# Initialize a list to hold the sequences
X_test_sequences = []

# Create the sequences
for i in range(num_sequences):
    sequence = X_test[i:i + time_steps]
    X_test_sequences.append(sequence)

# Convert the list to a numpy array
X_test = np.array(X_test_sequences)
y_test = y_test[:-time_steps+1]

In [ ]:
np.save('X_test.npy', X_test)
np.save('y_test.npy', y_test)

## ConvLSTM Model

### Model Architecture

In [ ]:
# Define the model
model = Sequential([

    # Add a ConvLSTM2D layer with 32 filters, a kernel size of (3, 3), and 'relu' activation
    layers.ConvLSTM2D(32, (5, 5), activation='relu', input_shape=(None, new_width, new_height, 3), return_sequences=True),
    layers.BatchNormalization(),
    # Use TimeDistributed to apply GlobalAveragePooling2D to each time step
    layers.TimeDistributed(GlobalAveragePooling2D()),

    # layers.ConvLSTM2D(32, (3, 3), activation='relu', return_sequences=True),
    # layers.BatchNormalization(),

    # Add a Dense layer with a single output neuron and 'sigmoid' activation (for binary classification)
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model with binary cross-entropy loss
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Print model summary
model.summary()

### Training

In [ ]:
# Define the number of epochs and other training parameters
epochs = 10  # Adjust as needed
batch_size = 32  # Adjust as needed

In [ ]:
y_train = np.asarray(y_train ).astype('float32').reshape((-1,1))
y_test = np.asarray(y_train ).astype('float32').reshape((-1,1))

In [ ]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

In [ ]:
# Train the model
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.2,
                    callbacks=[early_stopping, reduce_lr])

In [ ]:
# Loss plots
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy plots
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, 'r', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(train_loss) + 1)

## Evaluation

## Conclusion

* ...

# OLD

## Data Preparation

In [ ]:
def load_data(data_dir, df, series_len, image_shape):
    X = []  # List to store input sequences (image sequences)
    y = []  # List to store target labels
    n_pos = 0  # Number of positive samples
    n_neg = 0  # Number of negative samples

    for row in df.itertuples(index=False):
        filename = row.Timestamp + '.jpeg'
        label = row.Label

    # for index, row in df.iterrows():
    #     filename = row['Timestamp'] + '.jpeg'
    #     label = row['Label']

        if label not in ['N', 'P']:
            continue  # Skip entries with invalid labels

        if label == 'N':
          path_label = 'neg'
        else:
          path_label = 'pos'

        # Construct the full path to the image file
        image_path = os.path.join(data_dir, path_label, filename)

        # Read and preprocess the image
        image = cv2.imread(image_path)
        image = cv2.resize(image, image_shape)  # Resize the image to the desired shape
        image = image / 255.0  # Normalize pixel values to the range [0, 1]

        if len(X) < series_len:
            X.insert(0, image)  # Add the image to the input sequence

        if len(X) == series_len:
            X_arr = np.array(X)  # Convert the input sequence to a NumPy array
            X_arr = X_arr.reshape(series_len, *image_shape)  # Reshape to (series_len, height, width, channels)
            X.append(X_arr)  # Append the sequence to the list of input sequences
            if label == 'P':
                y.append(1)  # Positive label
                n_pos += 1
            else:
                y.append(0)  # Negative label
                n_neg += 1

        if len(X) > series_len:
            del X[0]  # Remove the oldest image from the sequence

    X = np.array(X)  # Convert the list of sequences to a NumPy array
    y = np.array(y)  # Convert the list of labels to a NumPy array
    nb = n_pos + n_neg  # Total number of samples

    return X, y, nb


In [ ]:
for row in train_label_csv.itertuples(index=False):
    filename = row.Timestamp
    label = row.Label

In [ ]:
for index, row in train_label_csv.iterrows():
    filename = row['Timestamp'] + '.jpeg'
    label = row['Label']

In [ ]:
filename, label #= '2011-03-25T114617.jpeg'


In [ ]:
image_path = os.path.join(train_dir_path, 'pos', filename)
image_path

In [ ]:
image = cv2.imread(image_path)
plt.imshow(image)

In [ ]:
 image = cv2.resize(image, image_shape)
 plt.imshow(image)

In [ ]:
series_len = 20
image_shape = (256, 256)

In [ ]:
X_train, y_train, nb_train = load_data(data_dir= train_dir_path,
                                       df = train_label_csv,
                                       series_len=series_len,
                                       image_shape=image_shape)

In [ ]:
X_test, y_test, nb_test = load_data(data_dir= test_dir_path,
                                    df = test_label_csv,
                                    series_len=series_len,
                                    image_shape=image_shape)

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

def load_and_preprocess_data(data_dir, df, series_len, image_shape, batch_size=32, shuffle_data=True):
    data = []
    labels = []

    if shuffle_data:
        df = shuffle(df)

    for index, row in df.iterrows():
        filename = row['Timestamp']
        label = row['Label']

        # Construct the full path to the image file
        image_path = os.path.join(data_dir, label, filename)

        # Read and preprocess the image
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, image_shape)  # Resize the image to the desired shape
            image = image / 255.0  # Normalize pixel values to the range [0, 1]

            data.append(image)
            if label == 'P':
                labels.append(1)  # Positive label
            else:
                labels.append(0)  # Negative label

    data = np.array(data)  # Convert the list of sequences to a NumPy array
    labels = np.array(labels)  # Convert the list of labels to a NumPy array

    return data, labels

# Example usage:
train_data, train_labels = load_and_preprocess_data(train_dir_path, train_label_csv, series_len, image_shape)
test_data, test_labels = load_and_preprocess_data(test_dir_path, test_label_csv, series_len, image_shape)


In [ ]:
train_data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def data_generator(data_dir, label_csv, batch_size, image_shape, shuffle=True):
    # Create an ImageDataGenerator with the desired preprocessing options
    datagen = ImageDataGenerator(
        rescale=1./255,  # Normalize pixel values to the range [0, 1]
        validation_split=0.2,  # Split data into training and validation sets
    )

    # Create data generators for the training and validation sets
    train_generator = datagen.flow_from_dataframe(
        dataframe=label_csv,
        directory=data_dir,
        x_col="Timestamp",  # Column containing the image filenames
        y_col="Label",     # Column containing the labels
        target_size=image_shape,  # Desired image shape
        batch_size=batch_size,
        class_mode="binary",  # Assumes binary classification (pos/neg)
        subset="training",  # Use the training subset
        shuffle=shuffle
    )

    validation_generator = datagen.flow_from_dataframe(
        dataframe=label_csv,
        directory=data_dir,
        x_col="Timestamp",
        y_col="Label",
        target_size=image_shape,
        batch_size=batch_size,
        class_mode="binary",
        subset="validation",  # Use the validation subset
        shuffle=shuffle
    )

    return train_generator, validation_generator


In [ ]:
series_len = 20
image_shape = (256, 256)
batch_size = 32

train_generator, validation_generator = data_generator(train_dir_path, train_label_csv, batch_size, image_shape, shuffle=True)

Make sure to replace '/path/to/data', '/path/to/labels.csv', '/path/to/train', '/path/to/train_labels.csv', '/path/to/test', and '/path/to/test_labels.csv' with the actual paths to your data directory, label CSV file, and subdirectories containing positive and negative samples.

This modified load_data function reads the image files, resizes them, normalizes pixel values, and constructs input sequences for the ConvLSTM model. It also reads labels from the CSV file and assigns binary labels (1 for positive, 0 for negative). Adjust the image_shape variable to match the desired image dimensions for your model.

## Build the ConvLSTM Model


Certainly! Here's a ConvLSTM model proposal based on the load_data function and the prediction task you mentioned earlier. This model will take a sequence of solar images as input and predict whether a solar flare will initiate a Coronal Mass Ejection (CME).

In [ ]:
from keras.models import Sequential
from keras.layers import ConvLSTM2D, MaxPooling2D, Flatten, Dense, Dropout

def convlstm_model(image_shape, series_len):
    model = Sequential()

    # ConvLSTM layer with 32 filters
    model.add(ConvLSTM2D(filters=32, kernel_size=(3, 3), input_shape=(series_len, *image_shape),
                         activation='relu', return_sequences=True))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # ConvLSTM layer with 64 filters
    model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3), activation='relu', return_sequences=True))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the output of ConvLSTM layers
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Dropout for regularization
    model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation

    return model



In [ ]:
# Define image dimensions and sequence length
image_shape = (512, 512)  # Adjust to your desired image dimensions
series_len = 20  # Adjust as needed

# Create the ConvLSTM model
model = convlstm_model(image_shape, series_len)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display model summary
model.summary()

In this proposed ConvLSTM model:

1. The model starts with a ConvLSTM2D layer with 32 filters and a kernel size of (3, 3). It takes input sequences of solar images with the specified shape and uses the ReLU activation function. This layer returns sequences of feature maps.

2. After each ConvLSTM2D layer, a MaxPooling2D layer is applied to downsample the spatial dimensions of the feature maps.

3. The output of the last ConvLSTM2D layer is flattened to create a one-dimensional feature vector.

4. Two fully connected (Dense) layers follow. The first Dense layer has 128 units with ReLU activation, and a dropout layer is added for regularization.

5. The final Dense layer has a single unit with a sigmoid activation function, which is suitable for binary classification tasks.

6. The model is compiled with binary cross-entropy loss and the Adam optimizer.

You can further fine-tune this model and adjust hyperparameters as needed for your specific task. Additionally, you can train the model using the training data and evaluate its performance on the testing data as mentioned in your original code.

## Train the model

In [ ]:
#type = 'convlstm'  # We're using ConvLSTM for the new prediction task
time_window = 12  # Adjust this as needed
train_again = 1  # Set to 1 to train the model again or 0 to load a pre-trained model
train_data_file = './normalized_training_' + str(time_window) + '.csv'
test_data_file = './normalized_testing_' + str(time_window) + '.csv'
result_file = './convlstm-' + str(time_window) + '-output.csv'
model_file = './convlstm-' + str(time_window) + '-model.h5'
start_feature = 4
mask_value = 0
series_len = 20
epochs = 20
batch_size = 256

In [ ]:
# Load or train the ConvLSTM model based on the new task
if train_again == 1:
    # # Load and preprocess training data
    # print('loading training data...')
    # X_train, y_train, nb_train = load_data(datafile=train_data_file,
    #                                         series_len=series_len,
    #                                         start_feature=start_feature,
    #                                         n_features=n_features,
    #                                         mask_value=mask_value,
    #                                         time_window=time_window)

    # Compute class weights for imbalance
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
    class_weight_ = {0: class_weights[0], 1: class_weights[1]}
    print('done loading training data...')

    # Create and train the ConvLSTM model
    # print('training the model, wait until it is finished...')
    # model = convlstm(n_features, series_len)
    # model.compile(loss='binary_crossentropy',
    #               optimizer='RMSprop',
    #               metrics=['accuracy'])
    history = model.fit(X_train,
                        y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=False,
                        shuffle=True,
                        class_weight=class_weight_)
    print('finished...')

    # Save the trained model
    model.save(model_file)
else:
    # Load a pre-trained ConvLSTM model
    print('loading model...')
    model = load_model(model_file)
    print('done loading...')


In this modified code:

1. The type is set to 'convlstm' to indicate the use of ConvLSTM for the new prediction task.

2. The convlstm function is a placeholder where you should define the architecture of your ConvLSTM model. You can design the ConvLSTM layers and any additional layers you need for your specific task.

3. The logic for loading data, training the model, and making predictions remains similar to the original code, with minor adjustments for the ConvLSTM model.

4. Ensure that you have the appropriate data files and adjust the time_window parameter according to your new task.

5. The output_result function should be updated to match the format of the new prediction task's output.

You'll need to define the ConvLSTM model architecture in the convlstm function, specifying the layers, activations, and any other hyperparameters required for your specific prediction task.

## Test the model

In [ ]:
# Test the model
print('loading testing data')
X_test, y_test, nb_test = load_data(datafile=test_data_file,
                                    series_len=series_len,
                                    start_feature=start_feature,
                                    n_features=n_features,
                                    mask_value=mask_value,
                                    time_window=time_window)
print('done loading testing data...')

# Predict and write results
print('predicting testing data...')
prob = model.predict(X_test,
                      batch_size=batch_size,
                      verbose=False,
                      steps=None)
print('done predicting...')

# Write prediction results into a file
print('writing prediction results into file...')
output_result(test_data_file=test_data_file,
              result_file=result_file,
              thresh=thresh)
print('done...')

## Evaluate the Model

# Graveyard

### try: ChatGPT

In [ ]:
# def load_data(datafile, flare_label, series_len, start_feature, n_features, mask_value):
#     df = pd.read_csv(datafile)
#     df_values = df.values
#     X = []
#     y = []
#     tmp = []
#     for k in range(start_feature, start_feature + n_features):
#         tmp.append(mask_value)
#     for idx in range(0, len(df_values)):
#         each_series_data = []
#         row = df_values[idx]
#         label = row[1][0]
#         if flare_label == 'C' and (label == 'X' or label == 'M'):
#             label = 'C'
#         if flare_label == 'C' and label == 'B':
#             label = 'N'
#         has_zero_record = False
#         # if at least one of the 25 physical feature values is missing, then discard it.
#         if flare_label == 'C':
#             if float(row[5]) == 0.0:
#                 has_zero_record = True
#             if float(row[7]) == 0.0:
#                 has_zero_record = True
#             for k in range(9, 13):
#                 if float(row[k]) == 0.0:
#                     has_zero_record = True
#                     break
#             for k in range(14, 16):
#                 if float(row[k]) == 0.0:
#                     has_zero_record = True
#                     break
#             if float(row[18]) == 0.0:
#                 has_zero_record = True

#         if has_zero_record is False:
#             cur_noaa_num = int(row[3])
#             each_series_data.append(row[start_feature:start_feature + n_features].tolist())
#             itr_idx = idx - 1
#             while itr_idx >= 0 and len(each_series_data) < series_len:
#                 prev_row = df_values[itr_idx]
#                 prev_noaa_num = int(prev_row[3])
#                 if prev_noaa_num != cur_noaa_num:
#                     break
#                 has_zero_record_tmp = False
#                 if flare_label == 'C':
#                     if float(row[5]) == 0.0:
#                         has_zero_record_tmp = True
#                     if float(row[7]) == 0.0:
#                         has_zero_record_tmp = True
#                     for k in range(9, 13):
#                         if float(row[k]) == 0.0:
#                             has_zero_record_tmp = True
#                             break
#                     for k in range(14, 16):
#                         if float(row[k]) == 0.0:
#                             has_zero_record_tmp = True
#                             break
#                     if float(row[18]) == 0.0:
#                         has_zero_record_tmp = True

#                 if len(each_series_data) < series_len and has_zero_record_tmp is True:
#                     each_series_data.insert(0, tmp)

#                 if len(each_series_data) < series_len and has_zero_record_tmp is False:
#                     each_series_data.insert(0, prev_row[start_feature:start_feature + n_features].tolist())
#                 itr_idx -= 1

#             while len(each_series_data) > 0 and len(each_series_data) < series_len:
#                 each_series_data.insert(0, tmp)

#             if len(each_series_data) > 0:
#                 X.append(np.array(each_series_data).reshape(series_len, n_features).tolist())
#                 y.append(label)
#     X_arr = np.array(X)
#     y_arr = np.array(y)
#     print(X_arr.shape)
#     return X_arr, y_arr


# def data_transform(data):
#     encoder = LabelEncoder()
#     encoder.fit(data)
#     encoded_Y = encoder.transform(data)
#     converteddata = np_utils.to_categorical(encoded_Y)
#     return converteddata

In [ ]:
# import os
# import numpy as np
# import cv2

# def load_images_from_folder(folder_path):
#     images = []
#     for filename in os.listdir(folder_path):
#         img = cv2.imread(os.path.join(folder_path, filename))
#         if img is not None:
#             images.append(img)
#     return images

# def create_dataset(data_folder, sequence_length, target_time_steps):
#     dataset = []
#     labels = []

#     for event_folder in os.listdir(data_folder):
#         event_path = os.path.join(data_folder, event_folder)
#         event_images = load_images_from_folder(event_path)

#         # Create sequences for the event images
#         event_sequences = create_sequences(event_images, sequence_length)

#         # Create labels for the event sequences
#         event_labels = create_labels(event_sequences, target_time_steps)

#         # Append the event sequences and labels to the main dataset
#         dataset.extend(event_sequences)
#         labels.extend(event_labels)

#     return np.array(dataset), np.array(labels)

# # Function to create sequences of AIA images with a fixed time duration
# def create_sequences(images, sequence_length):
#     sequences = []
#     num_images = len(images)
#     for i in range(0, num_images - sequence_length + 1, sequence_length):
#         sequence = images[i:i + sequence_length]
#         sequences.append(sequence)
#     return sequences

# # Function to create target labels for sequences
# def create_labels(sequences, target_time_steps):
#     labels = []
#     for sequence in sequences:
#         # Check if a CME occurs within the next target_time_steps
#         cme_occurs = any(sequence[i]['CME_occurs'] for i in range(target_time_steps))
#         labels.append(int(cme_occurs))
#     return labels

# # Assuming the AIA image data is organized in separate event folders within the 'data_folder'
# data_folder = '/path/to/data_folder'
# sequence_length = 6  # Choose the number of images in each sequence
# target_time_steps = 12  # Choose the time steps (intervals) for predicting CME occurrence

# # Create the dataset and labels
# dataset, labels = create_dataset(data_folder, sequence_length, target_time_steps)


### try 2: DS creation from Chat GPT

In [ ]:
data_dir = '00_Dataset/Liu2020_sampled/'

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_images_from_directory(directory):
    images = []
    timestamps = []
    for label in sorted(os.listdir(directory)):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for filename in sorted(os.listdir(label_dir)):
                if filename.endswith(".jpeg"):
                    img = load_img(os.path.join(label_dir, filename))
                    img_array = img_to_array(img)
                    images.append(img_array)
                    timestamp = filename.split(".")[0]  # Adjust the split based on your filename format
                    timestamps.append(timestamp)
    return images, timestamps

In [ ]:
os.listdir(data_dir)

In [ ]:
load_images_from_directory('00_Dataset/Liu2020_sampled/test/pos')

In [ ]:
def create_conv_lstm_dataset(data_dir, T, test_size=0.2):
    pos_dir = os.path.join(data_dir, 'train', 'pos')
    neg_dir = os.path.join(data_dir, 'train', 'neg')

    pos_images, pos_timestamps = load_images_from_directory(pos_dir)
    neg_images, neg_timestamps = load_images_from_directory(neg_dir)

    all_images = np.array(pos_images + neg_images)
    all_timestamps = pos_timestamps + neg_timestamps
    labels = np.array([1] * len(pos_images) + [0] * len(neg_images))

    X = []
    y = []

    for i in range(len(all_images) - T):
        sequence = all_images[i:i + T]
        label = labels[i + T]
        X.append(sequence)
        y.append(label)

    X = np.array(X)
    y = np.array(y)

    X_train, X_test, y_train, y_test, timestamps_train, timestamps_test = train_test_split(
        X, y, all_timestamps[T:], test_size=test_size, random_state=42
    )

    return X_train, y_train, X_test, y_test, timestamps_train, timestamps_test


In [ ]:
T = 20  # Number of time steps for the ConvLSTM sequence
X_train, y_train, X_test, y_test, timestamps_train, timestamps_test = create_conv_lstm_dataset(data_dir, T, test_size=0.2)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Define your ConvLSTM model using Keras here
# ...

# Compile and train the model
# model.compile(...)
# model.fit(...)


In [ ]:
# flare_label = sys.argv[1]
# train_again = int(sys.argv[2])
# filepath = './'
# n_features = 0
# if flare_label == 'C':
#   n_features = 14
# start_feature = 5
# mask_value = 0
# series_len = 10
# epochs = 7
# batch_size = 256
# nclass = 2
# result_file = './output.csv'

In [ ]:
# if train_again == 1:
#     # Train
#     X_train_data, y_train_data = load_data(datafile=filepath + 'normalized_training.csv',
#                                             flare_label=flare_label, series_len=series_len,
#                                             start_feature=start_feature, n_features=n_features,
#                                             mask_value=mask_value)

#     X_train = np.array(X_train_data)
#     y_train = np.array(y_train_data)
#     y_train_tr = data_transform(y_train)

#     class_weights = class_weight.compute_class_weight('balanced',
#                                                       np.unique(y_train), y_train)
#     class_weight_ = {0: class_weights[0], 1: class_weights[1]}
#     # print(class_weight_)

#     model = lstm(nclass, n_features, series_len)
#     model.compile(loss='categorical_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy'])

#     history = model.fit(X_train, y_train_tr,
#                         epochs=epochs, batch_size=batch_size,
#                         verbose=False, shuffle=True, class_weight=class_weight_)
#     model.save('./model.h5')
# else:
#     model = load_model('./model.h5')

#     # Test
#     X_test_data, y_test_data = load_data(datafile=filepath + 'normalized_testing.csv',
#                                          flare_label=flare_label, series_len=series_len,
#                                          start_feature=start_feature, n_features=n_features,
#                                          mask_value=mask_value)
#     X_test = np.array(X_test_data)
#     y_test = np.array(y_test_data)
#     y_test_tr = data_transform(y_test)

#     classes = model.predict(X_test, batch_size=batch_size, verbose=0, steps=None)

#     with open(result_file, 'w', encoding='UTF-8') as result_csv:
#         w = csv.writer(result_csv)
#         with open(filepath + 'normalized_testing.csv', encoding='UTF-8') as data_csv:
#             reader = csv.reader(data_csv)
#             i = -1
#             for line in reader:
#                 if i == -1:
#                     line.insert(0, 'Predicted Label')
#                 else:
#                     if classes[i][0] >= 0.5:
#                         line.insert(0, 'Positive')
#                     else:
#                         line.insert(0, 'Negative')
#                 i += 1
#                 w.writerow(line)

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, ConvLSTM2D, MaxPooling2D, Flatten, Dense

# # Assuming you have already created the 'dataset' and 'labels' using the provided code

# # Normalize the pixel values in the dataset to a range [0, 1]
# dataset = dataset.astype('float32') / 255.0

# # Reshape the dataset to match the input shape expected by ConvLSTM
# # Assuming the images have shape (image_height, image_width, num_channels)
# # and 'sequence_length' images per sequence
# sequence_length, image_height, image_width, num_channels = dataset.shape
# input_shape = (sequence_length, image_height, image_width, num_channels)

# # Build the ConvLSTM model
# model = Sequential()
# model.add(ConvLSTM2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Split the dataset into training and testing sets
# train_samples = int(0.8 * len(dataset))  # 80% for training
# x_train, y_train = dataset[:train_samples], labels[:train_samples]
# x_test, y_test = dataset[train_samples:], labels[train_samples:]

# # Train the ConvLSTM model
# epochs = 10  # Adjust the number of epochs based on your dataset and model complexity
# batch_size = 32
# model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))


### Setup Folder Structure for image sequences

We have two base folders
* training
* testing

In each of these are all the corresponding images stored which we downloaded from the `traing_samples_12` and `testing_samples_12` csv files. These resulted in
* XXXX images for training and
* XXXX images for testing

Howeever within the corresponding folder, 20 subsequent images belong to ONE event, which is the prediction task. Therefore, within the training and testing folders the images sequences are stored in their own folders, subsequently numberd. This leads to the following file structure:
* training
  * 0
  * 1
  * ...
  * XXX max number of training events -1
* testing
  * 0
  * 1
  * ...
  * XXX max number of testing events -1

In [ ]:
df_cme_training_12.shape[0] // 60

In [ ]:
# Path for events/ image sequences
path = working_directory + 'data/'
data_path_training = os.path.join(path + 'training')
data_path_testing = os.path.join(path + 'testing')

datasets = np.array(['training', 'testing'])
classes = np.array(['positive', 'negative'])

# Videos/sequences are going to be 20 frames/images in length
sequence_length = 20

# get total number of events/sequences
numb_training_sequences = df_cme_training_12.shape[0] //sequence_length
numb_testing_sequences = df_cme_test_12.shape[0] // sequence_length

# Folder start
start_folder = 30

In [ ]:
# # setup  folder structure
# for dataset in datasets:

#   for event_class in classes:
#       #dirmax = np.max(np.array(os.listdir(os.path.join(data_path_training, event_class))).astype(int))
#       for sequence in range(1,numb_training_sequences+1):
#           try:
#               os.makedirs(os.path.join(data_path_training, event_class, str(dirmax+sequence)))
#           except:
#               pass